# Граббинг данных из различных источников. создание сырых данных

## Портал «Наш Санкт-Петербург»
Портал создан по инициативе Губернатора Санкт-Петербурга Г.С.Полтавченко для оперативного взаимодействия жителей города с представителями органов власти Санкт-Петербурга.
<br>
<img src='../../../img/workshop_1/spb_portal.jpg' width="75%"/>
<br>
## Пример обращения
<img src='../../../img/workshop_1/event_example.jpg' width="75%"/>
<br>

## Карта событий 
<img src='../../../img/workshop_1/spb_event_map.jpg' width="75%"/>

По-профилировав немного трафик находим интересные эндпоинты:
- http://gorod.gov.spb.ru/public_api/v2/stats/problems/
- http://gorod.gov.spb.ru/public_api/v2/stats/classifier/

Напишем небольшой скрипт по скачиванию и сохранению данных с этих url.

In [1]:
from grabber import JsonGrabber, ProblemsGrabber
import transformators
import pandas as pd
import numpy as np
import os

from contextlib import contextmanager
from IPython.display import HTML, display
import json
import tabulate


from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

DATA_FOLDER = os.path.join('..','..','..', 'data', 'workshop_1')
json_grabber=JsonGrabber(DATA_FOLDER)

In [2]:
pr=json_grabber.get_problems()
types = json_grabber.get_types() 

### Пример того как выглядат полученные данные.
**с classifier рест ендпоинта**

In [3]:
print(json.dumps(types[0], indent=4, sort_keys=False,ensure_ascii=False))

{
    "id": 18,
    "name": "Бюджетное учреждение",
    "is_public": true,
    "categories": [
        {
            "id": 50,
            "name": "Нарушения в бюджетных образовательных  учреждениях",
            "is_public": true,
            "reasons": [
                {
                    "id": 160,
                    "name": "Отсутствие справочной информации на информационных стендах учреждений",
                    "is_public": true
                },
                {
                    "id": 159,
                    "name": "Неудовлетворительное санитарное состояние помещений учреждения",
                    "is_public": true
                }
            ]
        },
        {
            "id": 49,
            "name": "Нарушения в бюджетных учреждениях здравоохранения",
            "is_public": true,
            "reasons": [
                {
                    "id": 158,
                    "name": "Отсутствие справочной информации на информационных стендах учреждений",
 

**c problems рест ендпоинта**

In [4]:
print(json.dumps(pr[0], indent=4, sort_keys=False,ensure_ascii=False))
del pr

{
    "longitude": "30.3204382",
    "category": 10,
    "address": "г.Санкт-Петербург, улица Профессора Попова, дом 2, литера Б",
    "city_object": 1,
    "type_of": "building",
    "is_public": false,
    "district": 7,
    "url": null,
    "id": 824292,
    "municipality": 146,
    "status": "Модерация",
    "reason": 35,
    "building": 66093,
    "latitude": "59.9707241"
}


In [5]:
columns = ['id', 'name', 'is_public']
city_objects= pd.DataFrame(columns=columns)
categories = pd.DataFrame(columns=columns)
reasons = pd.DataFrame(columns=columns)

def get_dic(record):
    return {'id':record['id'],'name':record['name'],'is_public': int(record['is_public'])}

def append_record(df, f_get_dic, record):
    return df.append([f_get_dic(record)],ignore_index=False)

for k in types:
    city_objects = append_record(city_objects,get_dic,k)
    for cat in k['categories']:
        categories = append_record(categories,get_dic,cat)
        for reason in cat['reasons']:
            reasons = append_record(reasons,get_dic,reason)

def set_index(df):
    df=df.set_index('id')
    df=df.sort_index()
    df[['is_public']] = df[['is_public']].astype(int)
    return df
    
city_objects = set_index(city_objects)
categories = set_index(categories)
reasons = set_index(reasons)

In [6]:
def show_df(df):
    display(df.info())
    display(df.head())

def resolve_path(*path):
    return os.path.join(DATA_FOLDER,*path)

In [7]:
show_df(city_objects)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 1 to 20
Data columns (total 2 columns):
name         19 non-null object
is_public    19 non-null int32
dtypes: int32(1), object(1)
memory usage: 380.0+ bytes


None

,name,is_public
id,,
1,Дом,1
2,Двор,1
3,Улица,1
4,"Парк, сад, бульвар, сквер",1
5,Квартира,1


In [8]:
show_df(categories)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 1 to 66
Data columns (total 2 columns):
name         62 non-null object
is_public    62 non-null int32
dtypes: int32(1), object(1)
memory usage: 1.2+ KB


None

,name,is_public
id,,
1,Центральное отопление,1
2,Благоустройство,1
3,Благоустройство,1
4,Содержание МКД,1
5,Санитарное состояние,0


In [9]:
show_df(reasons)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253 entries, 1 to 257
Data columns (total 2 columns):
name         253 non-null object
is_public    253 non-null int32
dtypes: int32(1), object(1)
memory usage: 4.9+ KB


None

,name,is_public
id,,
1,Нарушено благоустройство после работы бригад,0
2,Низкая температура на лестничной площадке,1
3,Нарушение благоустройства после производства з...,1
4,Постоянное скопление безнадзорных животных,0
5,Мусор с контейнерной площадки своевременно не ...,0


In [10]:
city_objects.to_csv(resolve_path('csv','city_objects.csv'),index=True,encoding='utf-8')
categories.to_csv(resolve_path('csv','categories.csv'),index=True,encoding='utf-8')
reasons.to_csv(resolve_path('csv','reasons.csv'),index=True,encoding='utf-8')

In [11]:
problems=pd.read_json(resolve_path('json_sources','problems.json'),encoding='utf-8')
problems=problems.set_index('id')

In [12]:
show_df(problems)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 820028 entries, 824292 to 822
Data columns (total 13 columns):
address         566623 non-null object
building        566623 non-null float64
category        820028 non-null int64
city_object     820028 non-null int64
district        820028 non-null int64
is_public       820028 non-null bool
latitude        820028 non-null float64
longitude       820028 non-null float64
municipality    781088 non-null float64
reason          820028 non-null int64
status          820028 non-null object
type_of         820028 non-null object
url             400102 non-null object
dtypes: bool(1), float64(4), int64(4), object(4)
memory usage: 82.1+ MB


None

,address,building,category,city_object,district,is_public,latitude,longitude,municipality,reason,status,type_of,url
id,,,,,,,,,,,,,
824292,"г.Санкт-Петербург, улица Профессора Попова, до...",66093.0,10,1,7,False,59.970724,30.320438,146.0,35,Модерация,building,None
824291,None,NaN,3,3,7,False,59.968716,30.308962,146.0,54,Модерация,point,None
824290,None,NaN,3,3,36,False,60.008748,30.325581,78.0,78,Модерация,point,None
824289,"г.Санкт-Петербург, проспект Культуры, дом 29, ...",96138.0,15,5,10,False,60.049485,30.380597,90.0,48,Рассмотрение,building,None
824288,"г.Санкт-Петербург, бульвар Новаторов, дом 19, ...",91107.0,10,1,15,False,59.849157,30.264467,92.0,35,Рассмотрение,building,None


В результате парсинга мы получили справочника:
- типы городских объектов
- категории обращений
- причины обращений

и один датасет, собственно сами записи об обращениях граждн.

# Обработка полученых данных. 
## Очистка данных, заполнение пропусков, преобразование, генерация новых признаков

### Исслдедуем данные. 

In [13]:
problems=pd.DataFrame.from_csv(resolve_path('csv','problems_source.csv'))
display(problems.tail(4))

,address,building,category,city_object,district,is_public,latitude,longitude,municipality,reason,status,type_of,url
id,,,,,,,,,,,,,
826,"г.Санкт-Петербург, улица Типанова, дом 12, лит...",83641.0,2,2,14,True,59.851098,30.332273,133.0,8,Получен ответ,building,/facilities/83641/problems/
824,"г.Санкт-Петербург, улица Черняховского, дом 52...",130870.0,5,2,31,True,59.921309,30.359088,175.0,4,Получен ответ,building,/facilities/130870/problems/
823,"г.Санкт-Петербург, Лиговский проспект, дом 56,...",87619.0,17,4,31,False,59.923769,30.360044,175.0,11,Отклонено,building,NaN
822,NaN,NaN,3,3,6,False,59.951995,30.217189,76.0,15,Отклонено,point,NaN


In [14]:
@contextmanager
def show_complete_array(n=10):
    oldoptions = np.get_printoptions()
    np.set_printoptions(threshold=n)
    yield 
    np.set_printoptions(**oldoptions)

def print_df_inf(df):
    table = [(column,len(df[column].unique()),df[column].unique()) for column in df.columns]
    print("Клонки содержащие пустые значения: ", df.columns[df.isnull().any()].tolist())
    print("Всего строк: {:,}".format(len(df)))
    with show_complete_array():
        display(HTML(tabulate.tabulate(table, tablefmt='html',headers=['Имя параметра','Количество уникальных значений','Примеры уникальных значений'])))


print_df_inf(problems)

Клонки содержащие пустые значения:  ['address', 'building', 'municipality', 'url']
Всего строк: 820,028


Имя параметра,Количество уникальных значений,Примеры уникальных значений
address,34260,"['г.Санкт-Петербург, улица Профессора Попова, дом 2, литера Б' nan 'г.Санкт-Петербург, проспект Культуры, дом 29, корпус 1, литера А' ..., 'город Петергоф, Князевская улица, дом 22, литера А' 'г.Санкт-Петербург, улица Моисеенко, дом 43' 'город Пушкин, Новодеревенская улица, дом 16, литера А']"
building,34356,"[ 66093. nan 96138. ..., 126901. 74841. 54901.]"
category,62,"[10 3 15 ..., 14 30 6]"
city_object,19,"[ 1 3 5 ..., 12 16 7]"
district,18,"[ 7 36 10 ..., 38 32 13]"
is_public,2,[False True]
latitude,195824,"[ 59.9707241 59.968716 60.0087482 ..., 59.8510976 59.9213086 59.9519954]"
longitude,201030,"[ 30.3204382 30.3089618 30.3255808 ..., 30.359088 30.3600442 30.2171886]"
municipality,112,"[ 146. 78. 90. ..., 126. 103. nan]"
reason,251,"[35 54 78 ..., 21 1 27]"


## Что значит что объявление не публичное?

<img src='../../../img/workshop_1/not_access.jpg' width="75%" />

Для выявления какого то бизнесс велью, данных кажется не совсем достаточно, поищем данные в открытых источниках, для "обогощения" заявок с портала. Данные будем искать на портале открытых данных города Санкт-Петербург: http://data.gov.spb.ru.

Я нашел два дополнительных справочника дынных, который на мой взгляд, могли бы быть полезны в аналезе проблем с портала "Наш Санкт-Петербург": 

|Название данных | источник | название лок. файла| 
| ---| :--- :| ---: |
|Технико-экономические паспорта многоквартирных домов|http://data.gov.spb.ru/opendata/7840013199-passports_houses/|building_info_source.csv|
|Место розничной торговли или оказания услуг|http://wiki.openstreetmap.org/wiki/RU:Key:shop| shops.csv|

### Технико-экономические паспорта многоквартирных домов
Всего параметров 47, из них для нас представляю интерес колонки предоставляющие адрес дома, район к которому отностися дом, год постройки дома, года проведения капитального ремонта, количество этажей в доме и сколько людей проживает в доме(справочная информация).

In [15]:
buildings_info=pd.read_csv(resolve_path('csv','building_info_source.csv'))

C:\tools\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
buildings_info = buildings_info[['addr_street', 'addr_number',
       'addr_district', 'data_buildingdate','repair_year','data_storeys', 'data_residents']]

In [17]:
print_df_inf(buildings_info)
display(buildings_info.head(4))

Клонки содержащие пустые значения:  ['addr_street', 'addr_number', 'addr_district', 'data_buildingdate', 'repair_year', 'data_storeys', 'data_residents']
Всего строк: 21,523


Имя параметра,Количество уникальных значений,Примеры уникальных значений
addr_street,1478,"['Миллионная ул.' 'Коллонтай ул.' 'Подвойского ул.' ..., 'г.Сестрорецк, 9-я линия' 'пос.Левашово, Чкалова ул.' 'г.Пушкин, Полковая ул.']"
addr_number,1480,"['10' '25' '32' ..., '60 а' '14А' '39/13']"
addr_district,19,"['Центральный' 'Невский' 'Петроградский' ..., 'Курортный' 'Василеостровский' nan]"
data_buildingdate,355,"['1916' '1975' '1988' ..., '1859-1882' '1798-1858' '1985-1986']"
repair_year,8326,"['2000; 2011' '2007; 2008; 2016' '2012; 2012' ..., '1991; 1991; 2009' '2002; 2006; 2007; 2011; 2013; 2013; 2013; 2014' '2004; 2012; 2013; 2013; 2016']"
data_storeys,29,"[ 4. 9. 16. ..., 28. 21. nan]"
data_residents,1367,"[ 48. 355. 286. ..., 1209. 1096. 1863.]"


,addr_street,addr_number,addr_district,data_buildingdate,repair_year,data_storeys,data_residents
0,Миллионная ул.,10,Центральный,1916,2000; 2011,4.0,48.0
1,Коллонтай ул.,25,Невский,1975,2007; 2008; 2016,9.0,355.0
2,Подвойского ул.,32,Невский,1988,2012; 2012,16.0,286.0
3,Ленина ул.,34,Петроградский,1961,2005; 2006; 2006; 2007; 2007; 2008; 2009; 2009...,7.0,147.0


In [18]:
buildings_info=pd.read_csv(resolve_path('csv','buildings_info_transform.csv'),index_col=0)
buildings_info.head()

,addr_district,data_buildingdate,repair_year,data_storeys,data_residents,address,repair_count
0,Центральный,1916,2011,4.0,48.0,"Russian Federation, Saint Petersburg, Millionn...",2
1,Невский,1975,2016,9.0,355.0,"Russian Federation, Saint Petersburg, Kollonta...",3
2,Невский,1988,2012,16.0,286.0,"Russian Federation, Saint Petersburg, Podvoysk...",2
3,Петроградский,1961,2015,7.0,147.0,"Russian Federation, Saint Petersburg, Lenina S...",11
4,Кронштадский,1900,2014,2.0,29.0,"Russian Federation, Saint Petersburg, territor...",8


In [19]:
shops = pd.read_csv(resolve_path('csv','shops.csv'), encoding="UTF-8")
show_df(shops.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
shop    5 non-null object
name    5 non-null object
lat     5 non-null float64
lon     5 non-null float64
dtypes: float64(2), object(2)
memory usage: 240.0+ bytes


None

,shop,name,lat,lon
0,computer,Кей,60.048933,30.379963
1,photo,Яркий мир,60.043032,30.385281
2,supermarket,Магнит,60.046957,30.387773
3,supermarket,Магнит,60.043218,30.384391
4,supermarket,Народная 7Я семьЯ,60.012534,30.411242


In [20]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
import transformators

In [21]:
problems_source=pd.DataFrame.from_csv(resolve_path('csv','problems_source.csv'))
problems=problems_source.sample(n=int(len(problems_source)*.1), replace=True, random_state=42)
problems=problems.append(problems_source[problems_source['status']=='Не удовлетворён'])
print_df_inf(problems)

Клонки содержащие пустые значения:  ['address', 'building', 'municipality', 'url']
Всего строк: 82,003


Имя параметра,Количество уникальных значений,Примеры уникальных значений
address,17529,"['город Пушкин, Конюшенная улица, дом 37, литера А' 'г.Санкт-Петербург, проспект Героев, дом 24, корпус 3, литера А' 'г.Санкт-Петербург, Крюкова улица, дом 11, литера А' ..., 'г.Санкт-Петербург, улица Пограничника Гарькавого, дом 6, корпус 1, литера А' 'г.Санкт-Петербург, Остоумова улица, дом 21, литера А' 'г.Санкт-Петербург, Торфяная дорога, дом 2, корпус 1, литера А']"
building,17549,"[ 92194. 139731. 76648. ..., 19034. 198820. 124667.]"
category,61,"[19 8 10 ..., 51 44 6]"
city_object,19,"[ 1 2 3 ..., 7 9 16]"
district,18,"[35 16 11 ..., 10 13 50]"
is_public,2,[ True False]
latitude,41307,"[ 59.7209877 59.8612263 59.9618079 ..., 59.9879922 59.8641658 59.988555 ]"
longitude,41610,"[ 30.4156318 30.166544 30.4179586 ..., 30.3485164 30.3571933 30.2547917]"
municipality,112,"[ 160. 111. 105. ..., 126. 84. 103.]"
reason,241,"[210 26 35 ..., 90 256 160]"


In [ ]:
%%time
pipeline = make_pipeline(
        FunctionTransformer(transformators.delete_rows, kw_args=dict(column='type_of',value='point'),validate=False),
        FunctionTransformer(transformators.fill_none, validate=False),
        FunctionTransformer(transformators.columns_as_int, validate=False),
        FunctionTransformer(transformators.geocode_addres, validate=False),
        FunctionTransformer(transformators.search_near_shops, kw_args=dict(shops_info=shops), validate=False),
        FunctionTransformer(transformators.join_build_info,
                             kw_args=dict(building_info=buildings_info),
                            validate=False),
        FunctionTransformer(transformators.label_encoder, validate=False),
        FunctionTransformer(transformators.drop_columns, kw_args=dict(columns=['type_of', 'url']), validate=False)
    )
problems = pipeline.transform(problems)

In [ ]:
problems_pipe = problems.copy()
problems_pipe.to_csv(resolve_path('csv','problems_after_pipe.csv'),index=True,encoding='utf-8')

----------